In [ ]:
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
import seaborn as sns

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.dpi'] = 100

print("Libraries imported successfully")

## 1. Generate Synthetic AlphaFold Prediction Results

In [ ]:
# Create a realistic protein structure prediction
protein_id = "PROTEINX_ECOLI"
sequence = "MATSKGLAALLLQELQALPQVDVGSLALLLRQPTDDSGVAVSVLSQAKRGLAAVSQY" \
          "SRRSDYHDPPGSTRLVVMIGGHGKFTKQKGVLSDEQSASRESELKSRKKGLQPLFNKR"

protein_info = {
    'protein_id': protein_id,
    'sequence': sequence,
    'sequence_length': len(sequence),
    'organism': 'Escherichia coli',
    'uniprot_id': 'P0ABC1',
    'alphafold_version': 'v2.3.0',
    'prediction_timestamp': '2025-12-10T10:00:00Z',
    'msa_depth': 512
}

print(f"Protein: {protein_info['protein_id']}")
print(f"Length: {protein_info['sequence_length']} residues")
print(f"Organism: {protein_info['organism']}")
print(f"\nSequence (first 60 aa): {sequence[:60]}...")

## 2. Simulate Structural Predictions

In [ ]:
# Generate realistic pLDDT and structural features
np.random.seed(42)
seq_len = len(sequence)

# pLDDT (per-residue confidence): 0-100
# High in core, lower in loops
base_plddt = 75
plddt = base_plddt + np.cumsum(np.random.normal(0, 2, seq_len))
plddt = np.clip(plddt, 20, 95)

# Add known flexible regions
plddt[40:55] -= 15  # Flexible loop
plddt[85:100] -= 10  # C-terminal tail
plddt = np.clip(plddt, 20, 95)

# Secondary structure (simplified: H=helix, E=sheet, C=coil)
secondary_structure = np.random.choice(['H', 'E', 'C'], size=seq_len, p=[0.3, 0.25, 0.45])

# Hydrophobicity (Kyte-Doolittle scale, -4.5 to +4.5)
aa_hydro = {
    'A': 1.8, 'R': -4.5, 'N': -3.5, 'D': -3.5, 'C': 2.5,
    'Q': -3.5, 'E': -3.5, 'G': -0.4, 'H': -3.2, 'I': 4.5,
    'L': 3.8, 'K': -3.9, 'M': 1.9, 'F': 2.8, 'P': -1.6,
    'S': -0.8, 'T': -0.7, 'W': -0.9, 'Y': -1.3, 'V': 4.2
}

hydrophobicity = np.array([aa_hydro.get(aa, 0) for aa in sequence])

# Accessible Surface Area (ASA): 0-100 Ų per residue
# Correlated with hydrophobicity (exposed hydrophobic = problematic)
asa = 50 + 20 * (1 - hydrophobicity/5) + np.random.normal(0, 5, seq_len)
asa = np.clip(asa, 0, 100)

# Create residue-level dataframe
residue_df = pd.DataFrame({
    'position': np.arange(1, seq_len + 1),
    'amino_acid': list(sequence),
    'plddt': plddt,
    'secondary_structure': secondary_structure,
    'hydrophobicity': hydrophobicity,
    'asa': asa
})

print(f"Residue-level features extracted:")
print(residue_df.head(10))
print(f"\nSummary statistics:")
print(residue_df[['plddt', 'hydrophobicity', 'asa']].describe())

## 3. Identify Structural Features

In [ ]:
# Confidence classification
def classify_confidence(plddt_value):
    if plddt_value >= 90:
        return 'Very High'
    elif plddt_value >= 70:
        return 'High'
    elif plddt_value >= 50:
        return 'Medium'
    else:
        return 'Low'

residue_df['confidence'] = residue_df['plddt'].apply(classify_confidence)

# Confidence distribution
confidence_dist = residue_df['confidence'].value_counts()

print("\nConfidence Distribution:")
for conf in ['Very High', 'High', 'Medium', 'Low']:
    count = confidence_dist.get(conf, 0)
    pct = 100 * count / len(residue_df)
    print(f"  {conf:10s}: {count:3d} ({pct:5.1f}%)")

# Identify flexible regions (low pLDDT)
flexible_threshold = 60
flexible_regions = residue_df[residue_df['plddt'] < flexible_threshold]

print(f"\nFlexible Regions (pLDDT < {flexible_threshold}):")
if len(flexible_regions) > 0:
    print(f"  Total residues: {len(flexible_regions)}")
    print(f"  Mean pLDDT: {flexible_regions['plddt'].mean():.1f}")
    print(f"  Position ranges:")
    
    # Find contiguous regions
    positions = flexible_regions['position'].values
    gaps = np.diff(positions) > 1
    starts = np.concatenate([[positions[0]], positions[1:][gaps]])
    ends = np.concatenate([positions[:-1][gaps], [positions[-1]]])
    
    for start, end in zip(starts, ends):
        print(f"    Residues {start}-{end}")
else:
    print("  None found (structure is well-defined)")

## 4. Feature Extraction & Visualization

In [ ]:
# Create comprehensive feature visualization
fig, axes = plt.subplots(4, 1, figsize=(14, 10))

# Plot 1: pLDDT (confidence)
axes[0].bar(residue_df['position'], residue_df['plddt'], color='steelblue', alpha=0.7)
axes[0].axhline(y=90, color='green', linestyle='--', label='Very High (≥90)')
axes[0].axhline(y=70, color='orange', linestyle='--', label='High (70-90)')
axes[0].axhline(y=50, color='red', linestyle='--', label='Medium (50-70)')
axes[0].fill_between(residue_df['position'], 0, 100, where=(residue_df['plddt'] < 60), 
                      alpha=0.2, color='red', label='Flexible regions')
axes[0].set_ylabel('pLDDT Score')
axes[0].set_ylim([0, 100])
axes[0].set_title('Per-Residue Confidence (pLDDT)')
axes[0].legend(loc='upper right', fontsize=9)
axes[0].grid(True, alpha=0.3)

# Plot 2: Secondary Structure
ss_colors = {'H': 'red', 'E': 'blue', 'C': 'gray'}
ss_numeric = residue_df['secondary_structure'].map({'H': 3, 'E': 2, 'C': 1})
colors = [ss_colors[ss] for ss in residue_df['secondary_structure']]
axes[1].bar(residue_df['position'], ss_numeric, color=colors, alpha=0.7)
axes[1].set_ylabel('Secondary Structure')
axes[1].set_yticks([1, 2, 3])
axes[1].set_yticklabels(['Coil', 'Sheet', 'Helix'])
axes[1].set_title('Predicted Secondary Structure')
axes[1].grid(True, alpha=0.3)

# Plot 3: Hydrophobicity
colors_hydro = ['red' if h > 2 else 'blue' if h < -2 else 'gray' 
                for h in residue_df['hydrophobicity']]
axes[2].bar(residue_df['position'], residue_df['hydrophobicity'], color=colors_hydro, alpha=0.7)
axes[2].axhline(y=0, color='black', linestyle='-', linewidth=0.5)
axes[2].set_ylabel('Hydrophobicity')
axes[2].set_title('Amino Acid Hydrophobicity (Kyte-Doolittle)')
axes[2].grid(True, alpha=0.3)

# Plot 4: Accessible Surface Area
axes[3].plot(residue_df['position'], residue_df['asa'], marker='o', color='purple', alpha=0.6)
axes[3].fill_between(residue_df['position'], residue_df['asa'], alpha=0.3, color='purple')
axes[3].set_xlabel('Residue Position')
axes[3].set_ylabel('ASA (Ų)')
axes[3].set_title('Accessible Surface Area per Residue')
axes[3].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('resources/notebooks/alphafold-features.png', dpi=100, bbox_inches='tight')
plt.show()

print("Feature visualization saved")

## 5. Predict Protein Stability (Tm Estimation)

In [ ]:
# Estimate melting temperature from structure quality and composition

# Method 1: Based on mean pLDDT (proxy for overall stability)
mean_plddt = residue_df['plddt'].mean()
tm_from_plddt = 25 + (mean_plddt / 100) * 50  # Linear scaling: 25-75°C range

# Method 2: GC content approximation (Tm = 81.5 + 16.6*log[GC])
# Use hydrophobic core fraction as proxy
hydrophobic_core = np.mean(residue_df['hydrophobicity'] > 1)
tm_from_composition = 45 + hydrophobic_core * 15

# Ensemble estimate
tm_estimate = 0.6 * tm_from_plddt + 0.4 * tm_from_composition
tm_ci = (tm_estimate - 3, tm_estimate + 3)  # Approximate 95% CI

print(f"\nProtein Stability Estimation:")
print(f"  Mean pLDDT: {mean_plddt:.1f}")
print(f"  Tm from pLDDT: {tm_from_plddt:.1f}°C")
print(f"  Tm from composition: {tm_from_composition:.1f}°C")
print(f"  Ensemble Tm estimate: {tm_estimate:.1f}°C")
print(f"  95% CI: [{tm_ci[0]:.1f}, {tm_ci[1]:.1f}]°C")
print(f"\n  Recommended storage: -20°C to -80°C")
print(f"  Stability window: 2-8°C for ~48 hours")

## 6. Identify Epitope Candidates

In [ ]:
# Epitope prediction: flexible regions with exposed surface
epitope_candidates = []

# Sliding window analysis
window_size = 9
for i in range(len(residue_df) - window_size + 1):
    window = residue_df.iloc[i:i+window_size]
    
    # Criteria for epitope:
    # - Low mean pLDDT (flexible)
    # - High mean ASA (exposed)
    # - Not purely hydrophobic or hydrophilic
    
    mean_plddt_w = window['plddt'].mean()
    mean_asa_w = window['asa'].mean()
    mean_hydro_w = window['hydrophobicity'].mean()
    
    epitope_score = 0
    reasons = []
    
    # Low pLDDT = high flexibility
    if mean_plddt_w < 65:
        epitope_score += 2
        reasons.append('Flexible')
    
    # High ASA = exposed
    if mean_asa_w > 60:
        epitope_score += 2
        reasons.append('Exposed')
    
    # Balanced hydrophobicity
    if -1 < mean_hydro_w < 1:
        epitope_score += 1
        reasons.append('Balanced polarity')
    
    if epitope_score >= 3:
        epitope_candidates.append({
            'start': window.iloc[0]['position'],
            'end': window.iloc[-1]['position'],
            'score': epitope_score,
            'mean_plddt': mean_plddt_w,
            'mean_asa': mean_asa_w,
            'mean_hydro': mean_hydro_w,
            'reasons': ', '.join(reasons)
        })

epitope_df = pd.DataFrame(epitope_candidates).sort_values('score', ascending=False)

print(f"\nEpitope Candidate Identification:")
print(f"Candidates found: {len(epitope_df)}\n")

if len(epitope_df) > 0:
    for idx, row in epitope_df.head(5).iterrows():
        print(f"Region {row['start']}-{row['end']} (Score: {row['score']})")
        print(f"  pLDDT: {row['mean_plddt']:.1f}, ASA: {row['mean_asa']:.1f}, Hydro: {row['mean_hydro']:.2f}")
        print(f"  Reasons: {row['reasons']}")
        print()

## 7. Assay Optimization Suggestions

In [ ]:
# Generate actionable assay suggestions
suggestions = []

# Overall structure quality
if mean_plddt > 80:
    suggestions.append({
        'category': 'Structure Quality',
        'suggestion': 'High-confidence structure - suitable for structure-based design',
        'confidence': 'high'
    })
else:
    suggestions.append({
        'category': 'Structure Quality',
        'suggestion': 'Moderate structure confidence - recommend experimental validation',
        'confidence': 'medium'
    })

# Epitope design
if len(epitope_df) > 0:
    top_epitope = epitope_df.iloc[0]
    suggestions.append({
        'category': 'Antibody Design',
        'suggestion': f'Design epitope library targeting residues {int(top_epitope["start"])}-{int(top_epitope["end"])} (flexible, exposed region)',
        'confidence': 'high'
    })
else:
    suggestions.append({
        'category': 'Antibody Design',
        'suggestion': 'No obvious epitope candidates - consider whole-protein immunization',
        'confidence': 'medium'
    })

# Purification optimization
exposed_hydrophobic = np.mean((residue_df['asa'] > 60) & (residue_df['hydrophobicity'] > 1))
if exposed_hydrophobic > 0.1:
    suggestions.append({
        'category': 'Purification',
        'suggestion': f'High exposed hydrophobic surface ({exposed_hydrophobic*100:.0f}%) - recommend detergent (0.1-0.5 Triton X-100) or non-denaturing solubilization',
        'confidence': 'high'
    })
else:
    suggestions.append({
        'category': 'Purification',
        'suggestion': 'Hydrophobic surface low - standard aqueous buffers should suffice',
        'confidence': 'high'
    })

# Storage conditions
suggestions.append({
    'category': 'Storage',
    'suggestion': f'Estimated Tm = {tm_estimate:.1f}°C - store at -20°C or -80°C; avoid repeated freeze-thaw',
    'confidence': 'medium'
})

# QC recommendations
suggestions.append({
    'category': 'QC Assays',
    'suggestion': 'Recommend: SEC-MALS (oligomerization), DSF (stability), LCMS (intact mass), Thermal shift assay',
    'confidence': 'high'
})

print("\nASSAY OPTIMIZATION SUGGESTIONS:")
print("="*70)
for sugg in suggestions:
    print(f"\n{sugg['category'].upper()} ({sugg['confidence'].upper()})")
    print(f"  {sugg['suggestion']}")

print("\n" + "="*70)

## 8. QC Model Integration - Impurity Risk Prediction

In [ ]:
# Predict QC risk factors from structure

# Risk factors
risk_factors = {}

# 1. Aggregation risk (high exposed hydrophobic)
exposed_hydro_ratio = np.mean((residue_df['asa'] > 60) & (residue_df['hydrophobicity'] > 1))
risk_factors['aggregation_risk'] = min(1.0, exposed_hydro_ratio * 10)

# 2. Proteolytic cleavage risk (high flexibility at termini)
terminus_flexibility = np.mean(residue_df.iloc[:10]['plddt'] < 60) + \
                       np.mean(residue_df.iloc[-10:]['plddt'] < 60)
risk_factors['proteolysis_risk'] = terminus_flexibility / 2

# 3. Oxidation risk (methionine and tryptophan exposure)
oxidation_prone = sum(1 for aa in sequence if aa in ['M', 'W'])
risk_factors['oxidation_risk'] = min(1.0, (oxidation_prone / len(sequence)) * 5)

# 4. Disulfide bond reduction risk (cysteine count)
cysteine_count = sequence.count('C')
risk_factors['disulfide_risk'] = min(1.0, cysteine_count / 10)

# Overall quality score (inverse of risk)
avg_risk = np.mean(list(risk_factors.values()))
quality_score = 100 * (1 - avg_risk)

print(f"\nQC MODEL INTEGRATION - IMPURITY RISK ASSESSMENT")
print("="*70)
print(f"\nRisk Factor Scores (0-1 scale, 0=no risk, 1=high risk):")
print(f"  Aggregation risk: {risk_factors['aggregation_risk']:.3f}")
print(f"  Proteolysis risk: {risk_factors['proteolysis_risk']:.3f}")
print(f"  Oxidation risk: {risk_factors['oxidation_risk']:.3f}")
print(f"  Disulfide risk: {risk_factors['disulfide_risk']:.3f}")

print(f"\nOVERALL QUALITY SCORE: {quality_score:.1f}/100")

# Risk stratification
if quality_score > 80:
    risk_level = 'LOW'
    print(f"STATUS: {risk_level} - Expect high purity (>95%)")
elif quality_score > 60:
    risk_level = 'MODERATE'
    print(f"STATUS: {risk_level} - Recommend enhanced purification (SEC, IEC)")
else:
    risk_level = 'HIGH'
    print(f"STATUS: {risk_level} - Extensive purification required; consider mutation engineering")

print(f"\nRecommended QC Assays:")
if risk_factors['aggregation_risk'] > 0.3:
    print(f"  - Size-exclusion chromatography (SEC) - detect aggregates")
if risk_factors['proteolysis_risk'] > 0.3:
    print(f"  - LCMS intact mass - detect proteolytic fragments")
if risk_factors['oxidation_risk'] > 0.2:
    print(f"  - Native MS - detect oxidized forms")
if risk_factors['disulfide_risk'] > 0.1:
    print(f"  - Non-reducing SDS-PAGE - verify disulfide bonding state")
print(f"  - Capillary isoelectric focusing (cIEF) - heterogeneity screening")

## 9. Knowledge Graph Representation

In [ ]:
# Create knowledge graph (simplified JSON representation)
knowledge_graph = {
    'proteins': {
        protein_id: {
            'properties': {
                'sequence_length': protein_info['sequence_length'],
                'organism': protein_info['organism'],
                'uniprot_id': protein_info['uniprot_id']
            }
        }
    },
    'alphafold_models': {
        f"{protein_id}_v2.3.0": {
            'protein_id': protein_id,
            'model_version': protein_info['alphafold_version'],
            'metrics': {
                'mean_plddt': float(mean_plddt),
                'plddt_range': [float(residue_df['plddt'].min()), float(residue_df['plddt'].max())],
                'confidence_distribution': confidence_dist.to_dict()
            }
        }
    },
    'structural_features': {
        f"{protein_id}_features": {
            'protein_id': protein_id,
            'properties': {
                'mean_asa': float(residue_df['asa'].mean()),
                'hydrophobic_residue_fraction': float(np.mean(residue_df['hydrophobicity'] > 1)),
                'flexible_region_fraction': float(np.mean(residue_df['plddt'] < 60)),
                'predicted_tm': float(tm_estimate)
            }
        }
    },
    'epitope_candidates': {
        f"{protein_id}_epitopes": epitope_df.to_dict('records') if len(epitope_df) > 0 else []
    },
    'assay_suggestions': {
        f"{protein_id}_suggestions": suggestions
    },
    'qc_model_inputs': {
        f"{protein_id}_qc": {
            'protein_id': protein_id,
            'risk_factors': risk_factors,
            'quality_score': float(quality_score),
            'risk_level': risk_level
        }
    }
}

# Save to JSON
with open('resources/notebooks/alphafold-knowledge-graph.json', 'w') as f:
    json.dump(knowledge_graph, f, indent=2)

print("\nKNOWLEDGE GRAPH STRUCTURE:")
print(f"  Proteins: {len(knowledge_graph['proteins'])}")
print(f"  AlphaFold models: {len(knowledge_graph['alphafold_models'])}")
print(f"  Structural features: {len(knowledge_graph['structural_features'])}")
print(f"  Epitope candidates: {len(knowledge_graph['epitope_candidates'][f'{protein_id}_epitopes'])}")
print(f"  Assay suggestions: {len(knowledge_graph['assay_suggestions'][f'{protein_id}_suggestions'])}")
print(f"  QC model inputs: {len(knowledge_graph['qc_model_inputs'])}")
print(f"\nKnowledge graph saved to: resources/notebooks/alphafold-knowledge-graph.json")

## 10. Summary Report

In [ ]:
# Final summary
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# Plot 1: Risk factor radar
risk_names = list(risk_factors.keys())
risk_values = list(risk_factors.values())

axes[0, 0].barh(risk_names, risk_values, color=['red' if v > 0.5 else 'orange' if v > 0.3 else 'green' for v in risk_values])
axes[0, 0].set_xlim([0, 1])
axes[0, 0].set_xlabel('Risk Score')
axes[0, 0].set_title('QC Risk Factors')
axes[0, 0].grid(True, alpha=0.3, axis='x')

# Plot 2: Confidence distribution pie chart
conf_counts = residue_df['confidence'].value_counts()
colors_conf = {'Very High': 'green', 'High': 'lightgreen', 'Medium': 'orange', 'Low': 'red'}
colors_pie = [colors_conf.get(c, 'gray') for c in conf_counts.index]
axes[0, 1].pie(conf_counts.values, labels=conf_counts.index, autopct='%1.1f%%', colors=colors_pie)
axes[0, 1].set_title('pLDDT Confidence Distribution')

# Plot 3: Secondary structure composition
ss_counts = residue_df['secondary_structure'].value_counts()
ss_labels = {'H': 'Helix', 'E': 'Sheet', 'C': 'Coil'}
axes[1, 0].bar([ss_labels.get(k, k) for k in ss_counts.index], ss_counts.values, 
               color=['red', 'blue', 'gray'])
axes[1, 0].set_ylabel('Residue Count')
axes[1, 0].set_title('Secondary Structure Composition')
axes[1, 0].grid(True, alpha=0.3, axis='y')

# Plot 4: Summary metrics
axes[1, 1].axis('off')
summary_text = f"""
ALPHAFOLD → QC INTEGRATION SUMMARY

Protein: {protein_id}
Length: {protein_info['sequence_length']} aa

Structure Quality:
  Mean pLDDT: {mean_plddt:.1f}
  Confidence: {('Very High' if mean_plddt > 80 else 'High' if mean_plddt > 70 else 'Medium')}

Predicted Stability:
  Tm: {tm_estimate:.1f}°C [{tm_ci[0]:.1f}, {tm_ci[1]:.1f}]
  Storage: -20°C or -80°C

QC Assessment:
  Quality Score: {quality_score:.0f}/100
  Risk Level: {risk_level}
  Epitope Candidates: {len(epitope_df)}

Recommendation:
  {'✓ APPROVED for design' if quality_score > 80 else '⚠ RECOMMEND validation'}
"""
axes[1, 1].text(0.1, 0.5, summary_text, fontsize=10, family='monospace',
                verticalalignment='center', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.3))

plt.tight_layout()
plt.savefig('resources/notebooks/alphafold-qc-summary.png', dpi=100, bbox_inches='tight')
plt.show()

print("\n" + "="*70)
print("ALPHAFOLD → FEATURE EXTRACTION → QC INTEGRATION COMPLETE")
print("="*70)
print(f"\nProtein: {protein_id}")
print(f"Quality Score: {quality_score:.0f}/100 ({risk_level})")
print(f"Outputs saved:")
print(f"  - Feature visualization: resources/notebooks/alphafold-features.png")
print(f"  - QC summary: resources/notebooks/alphafold-qc-summary.png")
print(f"  - Knowledge graph: resources/notebooks/alphafold-knowledge-graph.json")
print("="*70)